# Transfer Learning part 1: Feature Extraction

_Transfer leaning is a process of leveraging a  working model's architecture and learning pattern for our own problems_

In [1]:
!pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
anyio                        4.10.0
argon2-cffi                  21.3.0
argon2-cffi-bindings         25.1.0
asttokens                    3.0.0
astunparse                   1.6.3
async-lru                    2.0.5
attrs                        25.4.0
babel                        2.16.0
beautifulsoup4               4.13.5
bleach                       6.2.0
brotlicffi                   1.1.0.0
certifi                      2023.5.7
cffi                         2.0.0
charset-normalizer           3.2.0
click                        8.1.7
colorama                     0.4.6
comm                         0.2.1
contourpy                    1.3.3
cycler                       0.12.1
debugpy                      1.8.16
decorator                    5.2.1
defusedxml                   0.7.1
executing                    2.2.1
fastjsonschema               2.20.0
flatbuffers                  25.9

In [2]:
!nvidia-smi

Thu Nov 13 17:49:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.83                 Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   37C    P8              4W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# download the data
import urllib.request
import zipfile

url = "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"
filename = "10_food_classes_10_percent.zip"

# Download the file
print(f"Downloading {filename}...")
urllib.request.urlretrieve(url, filename)
print("Download complete.")

# unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

Download complete.


In [4]:
#  how many images in each folder
import os

# Walk through 1 percent data dict
for dirpath, dirname, filename in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirname)} directories and {len(filename)} images in {dirpath}")

There are 2 directories and 0 images in 10_food_classes_10_percent
There are 10 directories and 0 images in 10_food_classes_10_percent\test
There are 0 directories and 250 images in 10_food_classes_10_percent\test\chicken_curry
There are 0 directories and 250 images in 10_food_classes_10_percent\test\chicken_wings
There are 0 directories and 250 images in 10_food_classes_10_percent\test\fried_rice
There are 0 directories and 250 images in 10_food_classes_10_percent\test\grilled_salmon
There are 0 directories and 250 images in 10_food_classes_10_percent\test\hamburger
There are 0 directories and 250 images in 10_food_classes_10_percent\test\ice_cream
There are 0 directories and 250 images in 10_food_classes_10_percent\test\pizza
There are 0 directories and 250 images in 10_food_classes_10_percent\test\ramen
There are 0 directories and 250 images in 10_food_classes_10_percent\test\steak
There are 0 directories and 250 images in 10_food_classes_10_percent\test\sushi
There are 10 directori

## create data loaders

In [5]:
# setup inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE=(224,224)
BATCH_SIZE = 32
train_dir="10_food_classes_10_percent/train"
test_dir="10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Traning data images")
# Use flow_from_directory to load images from a directory structure
train_data_10_percent=train_datagen.flow_from_directory(train_dir,
                                                       target_size=IMAGE_SHAPE,# Path to the directory containing subdirectories for each class
                                                       batch_size=BATCH_SIZE,
                                                       class_mode="categorical")

print("testing data images")
# Use flow_from_directory to load images from a directory structure
train_data_10_percent=train_datagen.flow_from_directory(test_dir,
                                                       target_size=IMAGE_SHAPE,# Path to the directory containing subdirectories for each class
                                                       batch_size=BATCH_SIZE,
                                                       class_mode="categorical")

Traning data images
Found 750 images belonging to 10 classes.
testing data images
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run while uor model train)

_creating a function for tf callback because we neet to create a new one for each_

In [6]:
import datetime

def create_tensorflow_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%y/%m/%d_%H:%M:%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"saving tensorflow log files to {log_dir}")
    return tensorboard_callback

In [7]:
!pip install kagglehub

  Using cached kagglehub-0.3.13-py3-none-any.whl.metadata (38 kB)
Using cached kagglehub-0.3.13-py3-none-any.whl (68 kB)


In [8]:
import kagglehub

# Download latest version
imagenet = kagglehub.model_download("google/efficientnet-v2/tensorFlow2/imagenet1k-b0-classification")
efficientnetv2_path = kagglehub.model_download("keras/efficientnetv2/keras/efficientnetv2_b0")

print("Path to model files:", efficientnetv2_path)
print("Path to model files:", imagenet)

Path to model files: C:\Users\updes\.cache\kagglehub\models\keras\efficientnetv2\keras\efficientnetv2_b0\2
Path to model files: C:\Users\updes\.cache\kagglehub\models\google\efficientnet-v2\tensorFlow2\imagenet1k-b0-classification\2


In [9]:
# import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

C:\Users\updes\anaconda3\envs\myenv\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [30]:
def create_model(model_url,num_classes):
    feature_extraction_layer = hub.KerasLayer(model_url,
                                              trainable=False,
                                              name="feature_extraction_layer",
                                              input_shape=IMAGE_SHAPE+(3,) #(224, 224, 3, 3)
                                             )
    model = tf.keras.Sequential([
        feature_extraction_layer,a
        layers.Dense(num_classes,activation="softmax",name="output_layer")
    ])
          
    return model

In [ ]:
def create_model(model_url,num_classes):
    feature_extraction_layer = hub.KerasLayer(model_url,
                                              trainable=False,
                                              name="feature_extraction_layer",
                                             )
    # 2. Define the model inputs
    inputs = tf.keras.Input(shape=IMAGE_SHAPE+(3,),
                            dtype="float32",
                            name="input_layer")

    # 3. Pass the inputs through the layers
    x = feature_extraction_layer(inputs)
    outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)
    
    # 4. Create the Keras Model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
          
    return model

In [31]:
IMAGE_SHAPE+(3,)

(224, 224, 3)

### creating and testing ResNet Tensorflow hub Features Extraction model

In [33]:
efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"
imagenet_model = create_model(efficientnet_url,
                              num_classes=train_data_10_percent.num_classes)

ValueError: Exception encountered when calling layer 'feature_extraction_layer' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'feature_extraction_layer' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, ragged=False, name=input_layer>
  • training=None

In [35]:
import tensorflow_hub as hub
m = tf.keras.Sequential([
    hub.KerasLayer(efficientnet_url)
])
m.build([None, 224, 224, 3])  # Batch input shape.

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x000002775A9F4590> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)